In [1]:
import core.utils as utils
import cv2
import json
from tqdm import tqdm
with open('./data/dataset/process/train.json','r',encoding='utf-8') as f:
    train_c = json.loads(f.read())

with open('./data/dataset/process/dev.json','r',encoding='utf-8') as f:
    dev_c = json.loads(f.read())

In [2]:
train_count = 0
for name in train_c:
    for index in train_c[name]:
         train_count+=1

dev_count = 0
for name in dev_c:
    for index in dev_c[name]:
         dev_count+=1

print(train_count)
print(dev_count)

54010
12817


In [3]:
"""
去除不使用的标签
1. 单句段落 x
2. 普通段落 x
3. 有框表格
4. 无框表格
5. 页眉
6. 页脚
7. 图片
8. 图表
9. 注脚
10. 公式
11. 目录
"""
o2n = {
    3:0,
    4:1,
    5:2,
    7:3,
    8:4,
    10:5,
    11:6
}

tqdm_ = tqdm(total=train_count)
train = {}
n2s = {}
for name in train_c:
    for index_ in train_c[name]:
        index = str(int(index_)-1)
        if not os.path.exists('/HDD/run/yolo_v3_tf2/yolo_v3_tf2.x/data/images/%s/%s.png'%(name, index)):
            continue
        img = cv2.imread('/HDD/run/yolo_v3_tf2/yolo_v3_tf2.x/data/images/%s/%s.png'%(name, index))
        path = './data/dataset/img_train/%s_%s.jpg'%(name, index_)
        label = []
        cache = set()
        for _ in [_['frame']+[o2n[_['type']]] for _ in train_c[name][index_] if _['type'] not in [1,2,6,9]]:
            y,x,h,w,cls = _
            if str([y,x,y+h,x+w,cls]) not in cache:
                label.append([y,x,y+h,x+w,cls])
                cache.add(str([y,x,y+h,x+w,cls]))
        if len(label)==0:
            continue
        train[path] = label
        cv2.imwrite(path, img)
        n2s[path]=img.shape
        tqdm_.update(1)

tqdm_.close()
tqdm_ = tqdm(total=dev_count)

with open('./data/n2s_train.json','w',encoding='utf-8') as f:
    f.write(json.dumps(n2s))
dev = {}
n2s = {}
for name in dev_c:
    for index_ in dev_c[name]:
        index = str(int(index_)-1)
        if not os.path.exists('/HDD/run/yolo_v3_tf2/yolo_v3_tf2.x/data/images/%s/%s.png'%(name, index)):
            continue
        img = cv2.imread('/HDD/run/yolo_v3_tf2/yolo_v3_tf2.x/data/images/%s/%s.png'%(name, index))
        path = './data/dataset/img_dev/%s_%s.jpg'%(name, index_)
        label = []
        cache = set()
        for _ in [_['frame']+[o2n[_['type']]] for _ in dev_c[name][index_] if _['type'] not in [1,2,6,9]]:
            y,x,h,w,cls = _
            if str([y,x,y+h,x+w,cls]) not in cache:
                label.append([y,x,y+h,x+w,cls])
                cache.add(str([y,x,y+h,x+w,cls]))
        if len(label)==0:
            continue
        dev[path] = label
        cv2.imwrite(path, img)
        n2s[path]=img.shape
        tqdm_.update(1)
tqdm_.close()

with open('./data/n2s.json','w',encoding='utf-8') as f:
    f.write(json.dumps(n2s))

 95%|█████████▌| 12195/12817 [04:18<00:13, 47.20it/s]


In [4]:
with open('./data/train.txt','w',encoding='utf-8') as f:
    for k in train:
        label = train[k]
        f.write('%s %s\n'%(k, ' '.join(["%d,%d,%d,%d,%d"%(_[0],_[1],_[2],_[3],_[4]) for _ in label])))

with open('./data/dev.txt','w',encoding='utf-8') as f:
    for k in dev:
        label = dev[k]
        f.write('%s %s\n'%(k, ' '.join(["%d,%d,%d,%d,%d"%(_[0],_[1],_[2],_[3],_[4]) for _ in label])))

In [5]:
import core.utils as utils
import numpy as np
import json

def read_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.loads(f.read())

def process(path, save_path, n2s):
    # 读取标注数据
    with open(save_path,'w',encoding='utf-8') as w:
        with open(path,'r',encoding='utf-8') as f:
            for line in f:
                line = line.strip().split(' ')
                p = line[0]
                tag = []
                shape  = n2s[p]
                for _ in line[1:]:
                    t = [float(__) for __ in _.split(',')]
                    if t[2]>shape[1] or t[3]>shape[0]:
                        continue
                    tag.append([float(__) for __ in _.split(',')])
                    tag[-1] = tag[-1][:-1]+[1]+tag[-1][-1:]


                # 进行一次极大值抑制
                if len(tag)==0:
                    continue
                tag_ = [_.tolist() for _ in utils.nms(np.array(tag),0.5)]
                if len(tag_)!=len(line[1:]):
                    print('%s:%d=>%d'%(p, len(tag), len(tag_)))
                w.write("%s %s\n"%(p, ' '.join(["%d,%d,%d,%d,%d"%(int(_[0]),int(_[1]),int(_[2]),int(_[3]),int(_[-1])) for _ in tag_])))

process('./data/train.txt','./data/train_.txt', read_json('./data/n2s_train.json'))
process('./data/dev.txt','./data/dev_.txt', read_json('./data/n2s.json'))

9.jpg:4=>3
./data/dataset/img_dev/6083015_187.jpg:7=>6
./data/dataset/img_dev/6083015_188.jpg:8=>6
./data/dataset/img_dev/6083015_195.jpg:5=>3
./data/dataset/img_dev/6083015_196.jpg:3=>2
./data/dataset/img_dev/6083015_197.jpg:6=>4
./data/dataset/img_dev/6083015_198.jpg:6=>3
./data/dataset/img_dev/6083015_199.jpg:5=>3
./data/dataset/img_dev/6083015_200.jpg:3=>2
./data/dataset/img_dev/6083015_201.jpg:3=>2
./data/dataset/img_dev/6083015_207.jpg:3=>2
./data/dataset/img_dev/6083015_211.jpg:3=>2
./data/dataset/img_dev/6083015_214.jpg:5=>3
./data/dataset/img_dev/6083015_216.jpg:5=>3
./data/dataset/img_dev/6083015_217.jpg:5=>3
./data/dataset/img_dev/6083015_218.jpg:4=>3
./data/dataset/img_dev/6083015_220.jpg:3=>2
./data/dataset/img_dev/6083015_224.jpg:3=>2
./data/dataset/img_dev/6083015_225.jpg:4=>3
./data/dataset/img_dev/6083015_227.jpg:5=>3
./data/dataset/img_dev/6083015_229.jpg:7=>4
./data/dataset/img_dev/6083015_233.jpg:3=>2
./data/dataset/img_dev/6083015_235.jpg:3=>2
./data/dataset/img_de

In [6]:
test = 1

In [1]:
import core.utils as utils
import cv2
path = "./data/dataset/img_dev/4502563_6.jpg" 
# tag = [[94, 101,1025,693,1,0], [1084,390,1095,400,1,3]]
tag_r = [[0.493703,0.9252,0.5,0.934105],[0.493184,0.924821,0.4974,0.933009]]


In [4]:
img = cv2.imread(path)
shape = img.shape
for i,_ in enumerate(tag_r):
    _ = [_[0]*shape[1],_[1]*shape[0],_[2]*shape[1],_[3]*shape[0],1,1]
    tag_r[i]=_
img = utils.draw_bbox(img, tag_r)
cv2.imwrite('./test.jpg',img)

True

In [5]:
tag_r

[[392.000182, 1038.9996, 397.0, 1048.9999149999999, 1, 1],
 [391.588096, 1038.573983, 394.9356, 1047.769107, 1, 1]]